available models:
1. en_ner_bc5cdr_md: https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_ner_bc5cdr_md-0.5.4.tar.gz
2. en_core_sci_md: https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_md-0.5.3.tar.gz
3. en_core_sci_sm: https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_sm-0.5.3.tar.gz
4. en_ner_bionlp13cg: https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_ner_bionlp13cg_md-0.5.4.tar.gz
5. en_ner_jnlpba_md: https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_ner_jnlpba_md-0.5.4.tar.gz
6. en_ner_craft_md:https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_ner_craft_md-0.5.4.tar.gz
7. en_core_sci_lg: https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_lg-0.5.4.tar.gz

8. For SciBERT use notebook sciBERT_WORKFLOW.ipynb
Swap in load_model and pip install install package step.

In [2]:
# install required model
pip install -q  https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_lg-0.5.4.tar.gz && pip install wordcloud && pip install fuzzywuzzy nltk --quiet && pip install transformers -U

  Using cached wordcloud-1.9.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
Using cached wordcloud-1.9.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (511 kB)
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.23.3-py3-none-any.whl.metadata (12 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached safetensors-0.4.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
Using cached huggingface_hub-0.23.3-py3-none-any.whl (401 kB)
Using cached safetensors-0.4.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
from imports import *

/opt/conda/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Load extractor class
extractor = Extractor()
# Load the model SELECT WHICH MODEL
extractor.load_model('en_core_sci_lg')
# Load the list of abtracts, select datasource, extractor
# set the number of abstracts and batch size
abstract_list_cleaned, abstract_ids, abstract_file_name = load_abstracts_clean_data_name('evaluation.abstracts', extractor, max_rows=1000, clean_data=True)
# Store the model name
model_name = extractor.get_model_name()
# Load annotations
annotation_list_cleaned = load_annotations('/root/DS_Evidence/ds_project_folder/data/raw_data/CHEMNER/evaluation.annotations.txt', abstract_ids, clean_data=True)

/opt/conda/lib/python3.10/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


Rows processed: 10

In [5]:
# Run performance metrics
num_common_elements, precision, recall, f1_score, annotation_unique_values, abstract_unique_values, intersection = calculate_evaluation_metrics(annotation_list_cleaned, abstract_list_cleaned)
# Print performance metrics
print_evaluation_metrics(annotation_list_cleaned, abstract_list_cleaned, abstract_ids)

Number of abstracts scanned: 1000
Number of entities extracted: 61209
Number of annotations in dataset: 8252
Number of common elements: 4276
Precision: 0.5181774115365972
Recall: 0.06985900766227189
F1 Score: 0.12311944832351968


In [6]:
# export entities from model
export_entities(model_name, abstract_unique_values, abstract_list_cleaned, abstract_file_name)
# append performance metrics
export_performance(model_name, abstract_list_cleaned, annotation_unique_values, abstract_unique_values, num_common_elements, precision, recall, f1_score,abstract_file_name)

In [7]:
# run fuzzy match and select character limit
filtered_matches, matches, char_diff = fuzzy_match(abstract_unique_values, annotation_unique_values, 10)

Rows processed: 61209
Total rows processed: 61209


In [8]:
# Calculate evaluation metrics with fuzzy matches
num_common_elements_fuzzy, precision_fuzzy, recall_fuzzy, f1_score_fuzzy = calculate_evaluation_metrics_with_fuzzy(annotation_list_cleaned, abstract_list_cleaned, filtered_matches)
# Print performance metrics
print_fuzzy_evaluation_metrics(annotation_list_cleaned, abstract_list_cleaned, filtered_matches, abstract_ids, abstract_unique_values,annotation_unique_values)

Number of abstracts scanned: 1000
Number of entities extracted: 61209
Number of annotations in dataset: 8252
Number of common elements: 6830
Precision: 0.8276781386330586
Recall: 0.11158489764577105
F1 Score: 0.19665711694332072


In [9]:
# export fuzzy match entities
export_entities_fuzzy(model_name, filtered_matches, abstract_list_cleaned, abstract_file_name)
# append performance metrics
export_performance_fuzzy(model_name, abstract_list_cleaned, annotation_unique_values, abstract_unique_values, num_common_elements_fuzzy, precision_fuzzy, recall_fuzzy, f1_score_fuzzy, char_diff, abstract_file_name)